In [1]:
import pydicom
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import threshold_otsu
from skimage.transform import resize
from collections import defaultdict
import pandas as pd
import os
from collections import defaultdict
import pickle
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [2]:
class ROI:
    
    def __init__(self, filename, border, character, flipped):
        dataset = pydicom.dcmread(filename)
        a = dataset.pixel_array
        
        if a.shape[0] < 1000:
            fl = list(filename)
            if fl[-5] == '0':
                fl[-5] = '1'
                filename = "".join(fl)
            else:
                fl[-5] = '0'
                filename = ''.join(fl)
                
        dataset = pydicom.dcmread(filename)
        a = dataset.pixel_array
        '''
        if "LEFT" in filename:
            self.mode = 'left'
        else:
            self.mode = 'right'
            a = a[:,::-1]
        '''
        if flipped:
            a = a[:, ::-1]
        
        self.filename = filename
        self.image = resize(a[:, :border], (227,227))
        self.character = character
        
    def plot_contour(self):
        if self.character == 'benign':
            plt.contour(self.image, cmap = 'plasma', linewidths = 0.5 )
        else:
            plt.contour(self.image, cmap = 'winter', linewidths = 0.5 )


class Photo:
    
    def find_border(self, x):
    
        '''
        arr = np.sum(x[1000:4000, :], axis = 0)
        i = 0
        while(arr[i] > 0):
            i += 1
        return i
        '''
        arr = np.sum(x[1000:4000, :], axis = 1)
        #plt.plot(range(len(arr)), arr)
        am = np.argmax(arr)
        #print(am)
        
        for i in range(x.shape[1] - 100 , 0 , -1):
            if x[1000 + am, i] > 0:
                return i
    
    def plot_image(self):
        
        plt.imshow(self.image, cmap = 'gray')
        
        for roi in  self.m_rois:
            roi.plot_contour()
            
        for roi in  self.b_rois:
            roi.plot_contour()
            
        plt.show()
            
    
    def __init__(self, filename, filenames_ROI_m, filenames_ROI_b):
        dataset = pydicom.dcmread(filename)
        a = dataset.pixel_array
        
        '''
        if "LEFT" in filename:
            self.mode = 'left'
        else:
            self.mode = 'right'
            a = a[:,::-1]
        '''

            
        self.filename = filename
        self.threshold = threshold_otsu(a)
        
        self.flipped = False
        
        temp = a > self.threshold
        if np.sum(temp[:, :temp.shape[1] // 2]) > np.sum(temp[:, temp.shape[1] // 2:]):
            self.mode = 'left'
        else:
            self.mode = 'right'
            a = a[:,::-1]
            self.flipped = True
        
        self.border = self.find_border(a > self.threshold)

        
        
        #plt.imshow(a > self.threshold, cmap = 'gray')
        
        self.image = resize(a[:, :self.border], (227,227))
        
        self.m_rois = []
        self.b_rois = []
        
        for fn in filenames_ROI_m:
            self.m_rois.append(ROI(fn, self.border, 'malignant', self.flipped))
        
        for fn in filenames_ROI_b:
            self.b_rois.append(ROI(fn, self.border, 'benign', self.flipped))
        

In [ ]:
filename = "CBIS-DDSM/Mass-Test_P_00016_LEFT_CC/10-04-2016-DDSM-30104/1-full mammogram images-14172/000000.dcm"
f1 = "CBIS-DDSM/Mass-Test_P_00016_LEFT_CC_1/10-04-2016-DDSM-09887/1-cropped images-26184/000001.dcm"
filenames_ROI_m = [f1]
filenames_ROI_b = []

ph = Photo(filename, filenames_ROI_m, filenames_ROI_b)
ph.plot_image()

In [ ]:
def cut_name(x):
    fst = x.find("/")
    scnd = fst + x[fst + 1:].find('/') + 1
    return x[fst+1:scnd]

In [ ]:
filename = "CBIS-DDSM/Mass-Test_P_00016_LEFT_CC/10-04-2016-DDSM-30104/1-full mammogram images-14172/000000.dcm"
f1 = "CBIS-DDSM/Mass-Test_P_00016_LEFT_CC_1/10-04-2016-DDSM-09887/1-cropped images-26184/000001.dcm"
filenames_ROI_m = [f1]
filenames_ROI_b = []

ph = Photo(filename, filenames_ROI_m, filenames_ROI_b)
ph.plot_image()